# Logistic Regression Multiple Features

## Normalisasi Data

In [2]:
#@title Install Go, `goimports` and GoNB code.

# Install Go and goimports.
!mkdir -p cache
!wget -q -O cache/go.tar.gz 'https://go.dev/dl/go1.22.0.linux-amd64.tar.gz'
!tar xzf cache/go.tar.gz
%env GOROOT=/content/go
!ln -sf "/content/go/bin/go" /usr/bin/go
!go version

# Install gonb, goimports, gopls.
!go install github.com/janpfeifer/gonb@latest
!ln -sf /root/go/bin/gonb /usr/bin/gonb
!go install golang.org/x/tools/cmd/goimports@latest
!ln -sf /root/go/bin/goimports /usr/bin/goimports
!go install golang.org/x/tools/gopls@latest
!ln -sf /root/go/bin/gopls /usr/bin/gopls

# Install gonb kernel configuration.
!gonb --install --logtostderr

# Python code to replace ipython kernel with GoNB.
import os
import colab_kernel_launcher

call_kernel_code='''
import os
import sys

connection_file = sys.argv[2]
os
try:
  os.execl("/usr/bin/gonb", "/usr/bin/gonb", "--kernel", connection_file)
finally:
  from ipykernel import kernelapp as app
  app.launch_new_instance()
'''
new_file = '/tmp/gonb_ipykernel_launcher.py'
old_file = colab_kernel_launcher.__file__
with open(new_file, 'w') as f:
  f.write(call_kernel_code)
os.replace(new_file, old_file)

# Finally kill current kernel to force restart.
import os; import sys; sys.stdout.flush(); os.kill(os.getpid(), 9)


Set: GOROOT="/content/go"
go version go1.22.0 linux/amd64


I0409 09:21:13.864462   11545 install.go:116] [a5395e22] Go (gonb) kernel configuration installed in "/root/.local/share/jupyter/kernels/gonb/kernel.json".


Cell[2]: Line 1 
 /tmp/gonb_6c390cd1/main.go:3:1: illegal character U+0023 '#'
 
package main

 #@title Install Go, `goimports` and GoNB code.
 
# Install Go and goimports.

ERROR: parsing go files in TempDir "/tmp/gonb_6c390cd1": /tmp/gonb_6c390cd1/main.go:3:1: illegal character U+0023 '#'

In [43]:
import "github.com/go-gota/gota/dataframe"

In [44]:
func getDataframe(file string) (dataframe.DataFrame, error) {
    var df dataframe.DataFrame
    f, err := os.Open(file)
    if err != nil {
        return df, err
    }
    defer f.Close()

    return dataframe.ReadCSV(f), nil
}

In [45]:
%%
df, err := getDataframe("Heart-disease.csv")
if err != nil {
    fmt.Println(err)
    return
}

fmt.Println(df)
fmt.Println(df.Select([]int{0,1,2,3,4}).Describe())
fmt.Println(df.Select([]int{5,6,7,8,9}).Describe())
fmt.Println(df.Select([]int{10,11,12}).Describe())

[303x14] DataFrame

    \ufeffage sex   cp    trestbps chol  fbs   restecg thalach exang oldpeak  ...
 0: 63        1     0     145      233   1     2       150     0     2.300000 ...
 1: 67        1     3     160      286   0     2       108     1     1.500000 ...
 2: 67        1     3     120      229   0     2       129     1     2.600000 ...
 3: 37        1     2     130      250   0     0       187     0     3.500000 ...
 4: 41        0     1     130      204   0     2       172     0     1.400000 ...
 5: 56        1     1     120      236   0     0       178     0     0.800000 ...
 6: 62        0     3     140      268   0     2       160     0     3.600000 ...
 7: 57        0     3     120      354   0     0       163     1     0.600000 ...
 8: 63        1     3     130      254   0     2       147     0     1.400000 ...
 9: 53        1     3     140      203   1     2       155     1     3.100000 ...
    ...       ...   ...   ...      ...   ...   ...     ...     ...   ...      

In [46]:
%%
minMax := []map[string]float64{
    map[string]float64{ "min" : 29.0, "max" : 77.0 },
    map[string]float64{ "min" : 0.0, "max" : 1.0 },
    map[string]float64{ "min" : 0.0, "max" : 3.0 },
    map[string]float64{ "min" : 94.0, "max" : 200.0 },
    map[string]float64{ "min" : 126.0, "max" : 564.0 },
    map[string]float64{ "min" : 0.0, "max" : 1.0 },
    map[string]float64{ "min" : 0.0, "max" : 2.0 },
    map[string]float64{ "min" : 71.0, "max" : 202.0 },
    map[string]float64{ "min" : 0.0, "max" : 1.0 },
    map[string]float64{ "min" : 0.0, "max" : 6.2 },
    map[string]float64{ "min" : 0.0, "max" : 2.0 },
    map[string]float64{ "min" : 0.0, "max" : 3.0 },
    map[string]float64{ "min" : 1.0, "max" : 4.0 },
}

// Open the loan dataset file.
f, err := os.Open("Heart-disease.csv")
if err != nil {
    log.Fatal(err)
}
defer f.Close()

// Create a new CSV reader reading from the opened file.
reader := csv.NewReader(f)
reader.FieldsPerRecord = 14

// Read in all of the CSV records
rawCSVData, err := reader.ReadAll()
if err != nil {
    log.Fatal(err)
}

// Create the output file.
f, err = os.Create("01-clean_heart.csv")
if err != nil {
    log.Fatal(err)
}
defer f.Close()

// Create a CSV writer.
w := csv.NewWriter(f)

// Sequentially move the rows writing out the parsed values.
for idx, record := range rawCSVData {

    // Skip the header row.
    if idx == 0 {
        // Write the header to the output file.
        if err := w.Write(record); err != nil {
            log.Fatal(err)
        }
        continue
    }

    // Initialize a slice to hold our parsed values.
    outRecord := make([]string, 14)

    for i, v := range record {
        if i == 13 {
            outRecord[i] = v
            continue
        }
        value, err := strconv.ParseFloat(v, 64)
        if err != nil {
            log.Fatal(err)
        }
        outRecord[i] = strconv.FormatFloat((value-minMax[i]["min"])/(minMax[i]["max"]-minMax[i]["min"]), 'f', 4, 64)
    }

    // Write the record to the output file.
    if err := w.Write(outRecord); err != nil {
        log.Fatal(err)
    }
}

// Write any buffered data to the underlying writer (standard output).
w.Flush()

if err := w.Error(); err != nil {
    log.Fatal(err)
}

## Membagi Data menjadi Training dan Test Set

In [47]:
%%
df, err := getDataframe("01-clean_heart.csv")
if err != nil {
  fmt.Println(err)
  return
}

trainingNum := (4 * df.Nrow()) / 5

// Shuffling data
allIdx := make([]int, df.Nrow())
for i := range allIdx {
  allIdx[i] = i
}
rand.Shuffle(len(allIdx), func(i, j int) { allIdx[i], allIdx[j] = allIdx[j], allIdx[i] })

// Gunakan indeks acak untuk split
trainingIdx := allIdx[:trainingNum]
testIdx := allIdx[trainingNum:]

trainingDF := df.Subset(trainingIdx)
testDF := df.Subset(testIdx)

setMap := map[int]dataframe.DataFrame{
    0: trainingDF,
    1: testDF,
}

for idx, setName := range []string{"01-train-heart.csv", "01-test-heart.csv"} {
    f, err := os.Create(setName)
    if err != nil {
        log.Fatal(err)
    }
    defer f.Close()

    w := bufio.NewWriter(f)
    // Set IncludeHeader to false
    if err := setMap[idx].WriteCSV(w, dataframe.WriteHeader(false)); err != nil {
        log.Fatal(err)
    }
}

## Menggunakan goml untuk Logistic Regression



In [48]:
import (
    "github.com/cdipaolo/goml/base"
    "github.com/cdipaolo/goml/linear"
    "gonum.org/v1/plot"
    "gonum.org/v1/plot/plotter"
    "gonum.org/v1/plot/plotutil"
    "github.com/janpfeifer/gonb/gonbui"
)

In [49]:
type ConfusionMatrix struct {
	positive      int
	negative      int
	truePositive  int
	trueNegative  int
	falsePositive int
	falseNegative int
	recall        float64
	precision     float64
	accuracy      float64
}

In [50]:
func (cm ConfusionMatrix) String() string {
	return fmt.Sprintf("\tPositives: %d\n\tNegatives: %d\n\tTrue Positives: %d\n\tTrue Negatives: %d\n\tFalse Positives: %d\n\tFalse Negatives: %d\n\n\tRecall: %.2f\n\tPrecision: %.2f\n\tAccuracy: %.2f\n",
		cm.positive, cm.negative, cm.truePositive, cm.trueNegative, cm.falsePositive, cm.falseNegative, cm.recall, cm.precision, cm.accuracy)
}

In [51]:
func tryValues(learningRate float64, regularization float64, iterations int, decisionBoundary float64, xTrain, xTest [][]float64, yTrain, yTest []float64) (*ConfusionMatrix, *linear.Logistic, error) {
	cm := ConfusionMatrix{}
	for _, y := range yTest {
		if y == 1.0 {
			cm.positive++
		}
		if y == 0.0 {
			cm.negative++
		}
	}

	// Instantiate and Learn the Model
	model := linear.NewLogistic(base.BatchGA, learningRate, regularization, iterations, xTrain, yTrain)
	model.Output = ioutil.Discard
	err := model.Learn()
	if err != nil {
		return nil, nil, err
	}

	// Evaluate the Model on the Test data
	for i := range xTest {
		prediction, err := model.Predict(xTest[i])
		if err != nil {
			return nil, nil, err
		}
		y := int(yTest[i])
		positive := prediction[0] >= decisionBoundary

		if y == 1 && positive {
			cm.truePositive++
		}
		if y == 1 && !positive {
			cm.falseNegative++
		}
		if y == 0 && positive {
			cm.falsePositive++
		}
		if y == 0 && !positive {
			cm.trueNegative++
		}
	}

	// Calculate Evaluation Metrics
	cm.recall = float64(cm.truePositive) / float64(cm.positive)
	cm.precision = float64(cm.truePositive) / (float64(cm.truePositive) + float64(cm.falsePositive))
	cm.accuracy = float64(float64(cm.truePositive)+float64(cm.trueNegative)) / float64(float64(cm.positive)+float64(cm.negative))
	return &cm, model, nil
}

In [52]:
%%
xTrain, yTrain, err := base.LoadDataFromCSV("01-train-heart.csv")
if err != nil {
    fmt.Println(err)
    return
}

xTest, yTest, err := base.LoadDataFromCSV("01-test-heart.csv")
if err != nil {
    fmt.Println(err)
    return
}

db := float64(0.5)
cm, model, err := tryValues(0.0001, 0.0, 1000, db, xTrain, xTest, yTrain, yTest)
if err != nil {
    fmt.Println(err)
    return
}

fmt.Printf("with Model: %s\n\n", model)
fmt.Printf("with Confusion Matrix:\n%s\n\n", cm)

with Model: h(θ,x) = 1 / (1 + exp(-θx))
θx = -0.810 + -0.07354(x[1]) + 0.24267(x[2]) + 0.25792(x[3]) + -0.02443(x[4]) + -0.10540(x[5]) + 0.00633(x[6]) + 0.12710(x[7]) + -0.93266(x[8]) + 0.95590(x[9]) + 0.47717(x[10]) + 0.42280(x[11]) + 1.00768(x[12]) + 0.98205(x[13])

with Confusion Matrix:
	Positives: 25
	Negatives: 36
	True Positives: 23
	True Negatives: 33
	False Positives: 3
	False Negatives: 2

	Recall: 0.92
	Precision: 0.88
	Accuracy: 0.92


